In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import DataLoader, Dataset
import json
import wandb

/root/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-05-09 20:50:35.431097: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64
2022-05-09 20:50:35.431215: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64
2022-05-09 20:50:35.431227: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing l

In [2]:
wandb.init(project="causal-lm", entity='cocoshe')


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cocoshe (use `wandb login --relogin` to force relogin)
2022-05-09 20:50:43.157548: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64
2022-05-09 20:50:43.157635: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64
2022-05-09 20:50:43.157645: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned 

In [3]:

json_data = []
with open('dataset/train.json', 'r',encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        json_data.append(data)
json_data[:10]

[{'content': '类型#裤*版型#宽松*风格#性感*图案#线条*裤型#阔腿裤',
  'summary': '宽松的阔腿裤这两年真的吸粉不少，明星时尚达人的心头爱。毕竟好穿时尚，谁都能穿出腿长2米的效果宽松的裤腿，当然是遮肉小能手啊。上身随性自然不拘束，面料亲肤舒适贴身体验感棒棒哒。系带部分增加设计看点，还让单品的设计感更强。腿部线条若隐若现的，性感撩人。颜色敲温柔的，与裤子本身所呈现的风格有点反差萌。'},
 {'content': '类型#裙*风格#简约*图案#条纹*图案#线条*图案#撞色*裙型#鱼尾裙*裙袖长#无袖',
  'summary': '圆形领口修饰脖颈线条，适合各种脸型，耐看有气质。无袖设计，尤显清凉，简约横条纹装饰，使得整身人鱼造型更为生动立体。加之撞色的鱼尾下摆，深邃富有诗意。收腰包臀,修饰女性身体曲线，结合别出心裁的鱼尾裙摆设计，勾勒出自然流畅的身体轮廓，展现了婀娜多姿的迷人姿态。'},
 {'content': '类型#上衣*版型#宽松*颜色#粉红色*图案#字母*图案#文字*图案#线条*衣样式#卫衣*衣款式#不规则',
  'summary': '宽松的卫衣版型包裹着整个身材，宽大的衣身与身材形成鲜明的对比描绘出纤瘦的身形。下摆与袖口的不规则剪裁设计，彰显出时尚前卫的形态。被剪裁过的样式呈现出布条状自然地垂坠下来，别具有一番设计感。线条分明的字母样式有着花式的外观，棱角分明加上具有少女元气的枣红色十分有年轻活力感。粉红色的衣身把肌肤衬托得很白嫩又健康。'},
 {'content': '类型#裙*版型#宽松*材质#雪纺*风格#清新*裙型#a字*裙长#连衣裙',
  'summary': '踩着轻盈的步伐享受在午后的和煦风中，让放松与惬意感为你免去一身的压力与束缚，仿佛要将灵魂也寄托在随风摇曳的雪纺连衣裙上，吐露出<UNK>微妙而又浪漫的清新之意。宽松的a字版型除了能够带来足够的空间，也能以上窄下宽的方式强化立体层次，携带出自然优雅的曼妙体验。'},
 {'content': '类型#上衣*材质#棉*颜色#蓝色*风格#潮*衣样式#polo*衣领型#polo领*衣袖长#短袖*衣款式#拼接',
  'summary': '想要在人群中脱颖而出吗？那么最适合您的莫过于这款polo衫短袖，采用了经典的polo领口和柔软纯棉面料，让您紧跟时尚

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
class MyDataset(Dataset):
    def __init__(self, input_data, tokenizer):
        super(MyDataset, self).__init__()
        self.input_data = input_data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.input_data)

    def __getitem__(self, idx):
        item = self.input_data[idx]
        content, summary = item['content'], item['summary']
        # con_sum = [content, summary]
        tokens_content = self.tokenizer(content, padding='max_length', max_length=128, truncation=True, return_tensors='pt', return_token_type_ids=True)
        tokens_summary = self.tokenizer(summary, padding='max_length', max_length=128, truncation=True, return_tensors='pt', return_token_type_ids=True)
        input_ids = tokens_content['input_ids'].squeeze(0)
        input_ids_summary = tokens_summary['input_ids'].squeeze(0)
        attention_mask = tokens_content['attention_mask'].squeeze(0)
        # print(input_ids.shape, token_type_ids.shape, attention_mask.shape)
        return input_ids, attention_mask, input_ids_summary

In [6]:
tokenizer = AutoTokenizer.from_pretrained("gpt2-chinese-cluecorpussmall")
model = AutoModelForCausalLM.from_pretrained("gpt2-chinese-cluecorpussmall").to(device)


In [7]:

token_test = tokenizer(["我是中国人", "中国人是我"], padding='max_length', max_length=128, truncation=True, return_tensors='pt', return_token_type_ids=True)
token_test.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [8]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(json_data, test_size=0.2, random_state=42, shuffle=True)
train_dataloader = DataLoader(MyDataset(train_data, tokenizer), batch_size=16, shuffle=True)
test_dataloader = DataLoader(MyDataset(test_data, tokenizer), batch_size=16, shuffle=True)

In [9]:
epochs = 5
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

In [10]:
def train():
    model.train()
    for epoch in range(epochs):
        for idx, batch in enumerate(tqdm(train_dataloader)):
            input_ids, attention_mask, input_ids_summary = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            input_ids_summary = input_ids_summary.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids_summary)
            loss = outputs[0]

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            wandb.log({'epoch': epoch, 'batch': idx, 'loss': loss.item()})

            if idx % 100 == 0:
                print('epoch: {}, loss: {}'.format(epoch, loss.item()))
        scheduler.step()

In [16]:
def evaluate():
    model.eval()
    with torch.no_grad():
        for idx, batch in enumerate(tqdm(test_dataloader)):
            input_ids, attention_mask, input_ids_summary = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            input_ids_summary = input_ids_summary.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids_summary)
            loss = outputs[0]
            print('loss: {}'.format(loss.item()))

In [12]:
train()

  0%|          | 1/5730 [00:02<4:37:48,  2.91s/it]

epoch: 0, loss: 9.473057746887207


  2%|▏         | 101/5730 [01:02<55:36,  1.69it/s] 

epoch: 0, loss: 5.210522651672363


  4%|▎         | 201/5730 [02:04<56:08,  1.64it/s]  

...

epoch: 4, loss: 4.67859411239624


 99%|█████████▉| 5701/5730 [57:42<00:17,  1.70it/s]

epoch: 4, loss: 4.703123569488525


  0%|          | 0/1433 [00:00<?, ?it/s]


ValueError: only one element tensors can be converted to Python scalars

In [17]:
evaluate()

  0%|          | 4/1433 [00:00<01:35, 14.98it/s]

loss: 4.835753440856934
loss: 5.1147565841674805
loss: 4.926013469696045
loss: 4.706615447998047


  0%|          | 6/1433 [00:00<01:54, 12.50it/s]

loss: 4.539798736572266
loss: 4.8711066246032715
loss: 4.873083114624023


  1%|          | 10/1433 [00:00<01:54, 12.42it/s]

...

loss: 4.595761775970459
loss: 4.96364688873291


100%|█████████▉| 1431/1433 [04:21<00:00,  5.86it/s]

loss: 4.916576385498047


100%|██████████| 1433/1433 [04:21<00:00,  5.48it/s]

loss: 5.216304779052734
loss: 4.705574035644531


In [13]:
torch.save(model.state_dict(), './model/gpt2_gen_ads.pth')